In [1]:
# 분야 2 | 유선 네트워크 장비의 신속한 조치를 위한 경보 유형 분류

# 아래의 코드는 다음과 같은 순서로 구성되어 있습니다.
# 1. 데이터 로드 및 전처리
# 2. 모델 학습 및 예측
# 3. 결과 제출
# 본 코드는 결과물 제출까지의 이해를 돕기위한 코드로, 반드시 이 구조를 따를 필요는 없습니다.
# 데이터 전처리, 모델의 구성 등에서 다양한 시도를 하시되, 올바른 채점을 위한 최종 결과물의 형태에 유의하시기 바랍니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

#데이터 전처리 및 모델

In [6]:
# 1. 데이터 로드
import pandas as pd

def ppr_data(df):
    # TODO: 데이터 전처리 코드 구현 ---------- #
    df = df.groupby('ticketno').apply(custom_info).reset_index()  # ticketno 기준 병합 및 전처리 수행

    x_df = df.iloc[:, :]
    return x_df

def custom_info(group):  # 임의로 작성된 전처리 코드입니다.
    d = {}
    group.sort_values(by='alarmtime', ascending=True, inplace=True)  # 경보 순서 정렬
    d['alarmmsg_original'] = ' '.join(group['alarmmsg_original'])  # 메시지 단순 병합
    if 'root_cause_type' in group.columns:  # 레이블 추출
        d['root_cause_type'] = group['root_cause_type'].iloc[0]  # 동일한 ticketno는 동일한 root_cause_type을 가짐
    else:
        d['root_cause_type'] = None  # 테스트 세트의 경우 정답 컬럼 없음
    return pd.Series(d, index=['alarmmsg_original', 'root_cause_type'])

train_df = pd.read_csv("/content/drive/MyDrive/제 2회 통신만 안정성 확보를 위한 인공지능 해커톤/Q2_data/Q2_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/제 2회 통신만 안정성 확보를 위한 인공지능 해커톤/Q2_data/Q2_test.csv")
train_df_filled=train_df.fillna(0)
test_df_filled=test_df.fillna(0)

x_train_df_filled = ppr_data(train_df_filled)
x_test_df_filled= ppr_data(test_df_filled)  # 테스트 세트의 경우 정답 컬럼 없음
x_train_df_filled

,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL ...,PowerFail
1,14771766.0,ETH-LINK-FAIL PSU-FAIL PSU-FAIL ETH-LINK-FAIL,PowerFail
2,14777089.0,PSU-FAIL PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL ...,PowerFail
3,14790052.0,PSU-FAIL PSU-FAIL,PowerFail
4,14879922.0,ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL PSU-FAIL ...,PowerFail
...,...,...,...
1109,21792877.0,OPT-LOS,LinkCut
1110,21793984.0,OPT-LOS OPT-LOS,LinkCut
1111,21799077.0,OPT-LOS OPT-LOS DCC-FAIL DCC-FAIL PW-LOC PW-LOC,LinkCut
1112,21809789.0,OPT-LOS OPT-LOS,LinkCut


In [7]:
x_train_df_filled['category_id'] = x_train_df_filled['root_cause_type'].factorize()[0]
from io import StringIO
category_id_df = x_train_df_filled[['root_cause_type', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['root_cause_type', 'category_id']].values)

In [8]:
x_test_df_filled
x_test_drop=x_test_df_filled.drop(columns=['root_cause_type'])
x_test_drop

,ticketno,alarmmsg_original
0,15238899.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL
1,15712444.0,PSU-FAIL PSU-FAIL
2,15723187.0,ETH-NO-RX-TRAFFIC PSU-FAIL PSU-FAIL
3,15737103.0,PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL
4,15737132.0,ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL PSU-FAIL
...,...,...
4322,22015278.0,DDM_RX_PWR_HIGH
4323,22015300.0,Loss Of Signal
4324,23818326.0,Server Signal Failure Loss Of Signal Server Si...
4325,23819373.0,Server Signal Failure


In [ ]:
result_train = x_train_df_filled["alarmmsg_original"].str.split()
data=result_train
flat_data = [item for sublist in data for item in sublist]
set_flat_data=set(flat_data)
print("train", set_flat_data)

# Use a set to keep track of unique values
result_test = x_test_drop["alarmmsg_original"].str.split()
data_test=result_test
flat_data_test = [item for sublist in data_test for item in sublist]
set_flat_data_test=set(flat_data_test)
print("test", set_flat_data_test)

# Find items in test set that are not in train set
items_in_test_not_in_train = set_flat_data_test - set_flat_data

# Find items in train set that are not in test set
items_in_train_not_in_test = set_flat_data - set_flat_data_test
items_in_test_not_in_train

In [10]:
import pandas as pd

df2 =  x_test_df_filled.drop(columns=['root_cause_type'])

# 'column1' 열의 값이 'apple'을 포함하는 행 추출
filtered_rows = df2[df2['alarmmsg_original'].str.contains('Unavailable')]

filtered_rows

,ticketno,alarmmsg_original
850,21866128.0,Improper Removal Improper Removal Loss Of Sign...
871,21866569.0,Loss Of Signal Improper Removal Underlying Res...
872,21866578.0,Loss Of Signal PPP Fail Improper Removal Under...
873,21866600.0,Loss Of Signal PPP Fail Improper Removal Under...
908,21867624.0,Loss Of Signal Improper Removal Underlying Res...
1024,21871865.0,Loss Of Signal Server Signal Failure Improper ...
1126,21876021.0,Improper Removal Underlying Resource Unavailab...
1129,21876049.0,Improper Removal Improper Removal Underlying R...
1769,21905075.0,PPP Fail Loss Of Signal Underlying Resource Un...
1770,21905147.0,Loss Of Signal PPP Fail Improper Removal Under...


In [14]:
import pandas as pd

df =  x_train_df_filled.drop(columns=['category_id', 'ticketno'])

#'root_cause_type' 열의 값이 'LinkCut'을 포함하는 행 추출
filtered_rows = df[df['root_cause_type'].str.contains('LinkCut')]
filtered_rows

,alarmmsg_original,root_cause_type
503,OPT-PWR-LOW OPT-PWR-LOW OPT-PWR-LOW OPT-PWR-LO...,LinkCut
504,LSP-LOC LSP-LOC LSP-LOC LSP-LOC LSP-LOC LSP-LO...,LinkCut
505,OPT-PWR-LOW OPT-PWR-LOW OPT-PWR-LOW OPT-PWR-LO...,LinkCut
506,OPT-LOS,LinkCut
507,OPT-LOS,LinkCut
...,...,...
1109,OPT-LOS,LinkCut
1110,OPT-LOS OPT-LOS,LinkCut
1111,OPT-LOS OPT-LOS DCC-FAIL DCC-FAIL PW-LOC PW-LOC,LinkCut
1112,OPT-LOS OPT-LOS,LinkCut


In [15]:
#'root_cause_type' 열의 값이 'UnitFall'을 포함하는 행 추출
filtered_rows2 = df[df['root_cause_type'].str.contains('UnitFail')]
filtered_rows2

,alarmmsg_original,root_cause_type
8,OPT-REMOVE OPT-LOS,UnitFail
9,NVRAM-FAIL,UnitFail
22,NVRAM-FAIL,UnitFail
23,OPT-REMOVE,UnitFail
41,OPT-REMOVE OPT-LOS OPT-REMOVE OPT-LOS,UnitFail
...,...,...
1024,ETH-LINK-FAIL AUTONEGO_MISMATCH AUTONEGO_MISMA...,UnitFail
1026,OPT-LOS OPT-LOS OPT-REMOVE,UnitFail
1027,OPT-LOS OPT-REMOVE OPT-REMOVE OPT-REMOVE OPT-LOS,UnitFail
1028,OPT-REMOVE OPT-REMOVE OPT-REMOVE,UnitFail


In [17]:
#'root_cause_type' 열의 값이 'SVS_TEMP_LOW'을 포함하는 행 추출
filtered_rows4 = df[df['alarmmsg_original'].str.contains('SYS_TEMP_LOW')]

In [18]:
id_to_category

{'PowerFail': 0, 'UnitFail': 1, 'LinkCut': 2}

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 8))

features = tfidf.fit_transform(x_train_df_filled.alarmmsg_original).toarray()
labels = x_train_df_filled.category_id
features.shape

(1114, 522)

In [20]:
text_features = tfidf.transform(x_test_drop['alarmmsg_original'])

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [23]:
clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = {'criterion': ['gini'],
               'max_depth': [12],
               'max_features': ['sqrt'],
               'min_samples_leaf': [4],
               'min_samples_split': [7],
               'n_estimators': [20]}, n_iter = 10,
                           cv = 5, verbose= 1, random_state= 101, n_jobs = -1)
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, x_train_df_filled.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_valid_pred = model.predict(text_features)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [24]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.047287,0.002265,0.002908,0.000299,20,7,4,sqrt,12,gini,"{'n_estimators': 20, 'min_samples_split': 7, '...",0.966667,0.979866,0.966443,0.95302,0.979866,0.969172,0.010033,1


#후처리

In [25]:
id_to_category = {
    0: 'PowerFail',
    1: 'UnitFail',
    2: 'LinkCut'
}

In [26]:
len(y_valid_pred)

4327

In [27]:
id_to_category = {
    0: 'PowerFail',
    1: 'UnitFail',
    2: 'LinkCut'
}

In [30]:
id_to_root_cause = {
    0: "PowerFail",
    1: "UnitFail",
    2: "LinkCut",
    # Add more mappings if needed
}
predicted_root_cause = [id_to_root_cause[cat_id] for cat_id in y_valid_pred]

# Print the predictions with root_cause_type
print(predicted_root_cause[:10])

['PowerFail', 'PowerFail', 'PowerFail', 'PowerFail', 'PowerFail', 'UnitFail', 'PowerFail', 'PowerFail', 'PowerFail', 'UnitFail']


In [31]:
x_test_df_filled
x_test_drop=x_test_df_filled.drop(columns=['root_cause_type'])
x_test_drop

,ticketno,alarmmsg_original
0,15238899.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL
1,15712444.0,PSU-FAIL PSU-FAIL
2,15723187.0,ETH-NO-RX-TRAFFIC PSU-FAIL PSU-FAIL
3,15737103.0,PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL
4,15737132.0,ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL PSU-FAIL
...,...,...
4322,22015278.0,DDM_RX_PWR_HIGH
4323,22015300.0,Loss Of Signal
4324,23818326.0,Server Signal Failure Loss Of Signal Server Si...
4325,23819373.0,Server Signal Failure


In [32]:
x_test_drop['root_cause_type']=predicted_root_cause
x_test_drop

,ticketno,alarmmsg_original,root_cause_type
0,15238899.0,PSU-FAIL ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL,PowerFail
1,15712444.0,PSU-FAIL PSU-FAIL,PowerFail
2,15723187.0,ETH-NO-RX-TRAFFIC PSU-FAIL PSU-FAIL,PowerFail
3,15737103.0,PSU-FAIL ETH-LINK-FAIL ETH-LINK-FAIL PSU-FAIL,PowerFail
4,15737132.0,ETH-LINK-FAIL PSU-FAIL ETH-LINK-FAIL PSU-FAIL,PowerFail
...,...,...,...
4322,22015278.0,DDM_RX_PWR_HIGH,UnitFail
4323,22015300.0,Loss Of Signal,UnitFail
4324,23818326.0,Server Signal Failure Loss Of Signal Server Si...,UnitFail
4325,23819373.0,Server Signal Failure,UnitFail


In [33]:
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('BATTERY_FAIL'), 'root_cause_type'] = 'PowerFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('BATT_ENV_FAIL'), 'root_cause_type'] = 'PowerFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('UNIT_FAIL'), 'root_cause_type'] = 'UnitFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('Loss Of Signal'), 'root_cause_type'] = 'LinkCut'

In [34]:
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('OTP'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('LSP-LOC'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('ETHER_LINK_DOWN'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('FAIL OPT'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('ERR ETH'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('NE Not Reachable'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('MEP_PWE_LOC'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('OAM'), 'root_cause_type'] = 'LinkCut'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('Rx Fail'), 'root_cause_type'] = 'LinkCut'

x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('REMOVE OPT'), 'root_cause_type'] = 'UnitFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('REMOVE ETH'), 'root_cause_type'] = 'UnitFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('SFP'), 'root_cause_type'] = 'UnitFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('Input Power Degrade'), 'root_cause_type'] = 'UnitFail'


x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('FAIL PSU'), 'root_cause_type'] = 'PowerFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('FAIL ETH'), 'root_cause_type'] = 'PowerFail'
x_test_drop.loc[x_test_drop['alarmmsg_original'].str.contains('48V-FAIL'), 'root_cause_type'] = 'PowerFail'

In [35]:
x_test_drop.drop('alarmmsg_original', axis=1, inplace=True)

In [36]:
x_test_drop

,ticketno,root_cause_type
0,15238899.0,PowerFail
1,15712444.0,PowerFail
2,15723187.0,PowerFail
3,15737103.0,PowerFail
4,15737132.0,PowerFail
...,...,...
4322,22015278.0,UnitFail
4323,22015300.0,LinkCut
4324,23818326.0,LinkCut
4325,23819373.0,UnitFail


#결과 저장

In [37]:
# 3. 결과 제출
# 본 코드는 제출되는 파일의 형태에 대한 가이드로, 반드시 아래 구조를 따를 필요 없이 자유롭게 코드를 작성해도 무방합니다.
# 제출 포맷에 대해선 data/Q2_label_sample.csv를 참조하세요.
#
# 분야 2의 경우, 전표(ticket) 하나에 하나의 근원장애(root_cause_type)을 매칭해야 합니다.
#   주의: 경보(alarm) 개수와 전표(ticket) 개수는 다르며, 예측할 대상은 전표입니다.
#   주의: ticketno 컬럼 기준으로 오름차순 정렬이 필요합니다.
# 분야 2의 제출 파일은 2개 컬럼 [ticketno, root_cause_type]을 가져야 합니다.

def submitResult(pred):
    try:
        label = pd.read_csv('/content/drive/MyDrive/제 2회 통신만 안정성 확보를 위한 인공지능 해커톤/Q2_data/Q2_label_sample.csv')
        # ticketno 순서와 개수가 일치하는지 확인
        if (label['ticketno'] == pred['ticketno']).all():
            print("Check: ticketno 순서와 샘플 수가 일치합니다.")
        else:
            print("Warning: 테스트 세트와 모델 예측의 ticketno가 일치하지 않습니다.")
            return

        pred.to_csv('Q2_submitResult.csv', index=False)
        print("Done : Q2_submitResult.csv 파일로 저장되었습니다.")
    except Exception as e:
        # 예외가 발생한 경우 오류 메시지 출력
        print("Error:", e)

submitResult(x_test_drop)

Check: ticketno 순서와 샘플 수가 일치합니다.
Done : Q2_submitResult.csv 파일로 저장되었습니다.
